# Assignment 1

In [1]:
target_row = 10
target_column = 7
target_part = "A"
print(f"{target_row} {target_column} {target_part}")

5 1 A


In [2]:
import pandas as pd
import math
from scipy import stats

In [3]:
xls = pd.ExcelFile('Lab 1 Data.xlsx')

In [4]:
xlsx_rand_num = pd.read_excel(xls, 'Random Number Table', header=0, names=['Row', 'Column', 'Rand', 'A', 'B', 'C'])

In [5]:
xlsx_population = pd.read_excel(xls, 'Population', header=None, names=['ID', 'Gender', 'Score'])

In [6]:
indexed_data = pd.merge(left=xlsx_rand_num, right=xlsx_population, left_on=target_part, right_on="ID")

In [7]:
data = indexed_data.loc[((indexed_data['Row'] == target_row) & (indexed_data['Column'] == target_column)).idxmax():] \
    .drop_duplicates(subset=["Gender", "Score"]).head(60)

In [8]:
c,m,s = data['Score'].describe()[0:3]

In [9]:
ci1_high = m + 1.96 * s / math.sqrt(c)
ci1_low = m - 1.96 * s / math.sqrt(c)

### 95% CI for all values.

In [10]:
print(f"Count={c:.2f}, Mean={m:.2f}, SD={s:.2f}, Int-High={ci1_high:.2f}, Int-Low={ci1_low:.2f}")

Count=60.00, Mean=50.74, SD=9.24, Int-High=53.08, Int-Low=48.40


### 90% CI for Male and Female (note the DF changes per group)

In [11]:
data_agg = data.groupby('Gender')['Score'].agg(["count", "mean", "std", "sem"]).reset_index()

In [12]:
ci95_hi = []
ci95_low = []

In [13]:
#1.706 = t @ df 26
for i in data_agg.index:
    g, c, m, s, se = data_agg.loc[i]
    ci95_hi.append(m + stats.t.ppf(1-.05, c) * s / math.sqrt(c))
    ci95_low.append(m - stats.t.ppf(1-.05, c) * s / math.sqrt(c))

In [14]:
data_agg['ci95_low'] = ci95_low
data_agg['ci95_hi'] = ci95_hi

In [15]:
data_agg

Gender  count       mean       std       sem   ci95_low    ci95_hi
0      F     27  51.292593  9.484235  1.825242  48.183679  54.401506
1      M     33  50.287879  9.166336  1.595654  47.587457  52.988301

### 99% CI for proportion of females out of 60.

In [16]:
female_count = data_agg.query("Gender == 'F'")['count'].sum()
total_count = data.shape[0]
p = female_count / total_count

In [17]:
p

np.float64(0.45)

In [18]:
se = math.sqrt(p * (1-p)/60)

In [19]:
p_ci_low = p - 2.576 * se  # 2.576 is z @ 99%

In [20]:
p_ci_high = p + 2.576 * se

In [21]:
p_ci_low, p_ci_high

(np.float64(0.2845534043867931), np.float64(0.6154465956132069))